In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import data_loader
import datetime
import pickle
import timeit
import time
import os
from utils import *
tf.keras.backend.set_floatx('float32')

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.set_visible_devices(physical_devices[3],'GPU')
os.makedirs('./igan_samples', exist_ok=True)

2023-10-09 23:01:18.131364: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-09 23:01:18.209803: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 23:01:19.476979: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
(l,l) = (8,8)
lattice_size  = 8

In [3]:
def sample_generation(no_samples,temp,lattice_size = 4):
    zrand   = tf.random.normal([no_samples,lattice_size,lattice_size,1],mean=0.0,stddev=1.0,dtype=tf.dtypes.float32)
    tz      = tf.cast(tf.repeat(temp,no_samples*lattice_size*lattice_size),tf.float32)
    tz      = tf.reshape(tz,[no_samples,lattice_size,lattice_size,1])
    z       = tf.concat([zrand,tz],axis=-1)
    samples = generator(z)
    lattices= np.arctan2(samples[:,:,:,1],samples[:,:,:,0])/(2*np.pi) 
    return np.reshape(lattices,(-1,lattice_size,lattice_size,1))

In [4]:
def generalised_sample_generation(model,no_samples,temp,lattice_size=(4,4)):
    zrand   = tf.random.normal([no_samples,lattice_size[0],lattice_size[1],1],mean=0.0,stddev=1.0,dtype=tf.dtypes.float32)
    tz      = tf.cast(tf.repeat(temp,no_samples*lattice_size[0]*lattice_size[1]),tf.float32)
    tz      = tf.reshape(tz,[no_samples,lattice_size[0],lattice_size[1],1])
    z       = tf.concat([zrand,tz],axis=-1)
    samples = model(z)
    return samples

In [5]:
def samples_generation_for_all_temp(no_samples,temp_array,lattice_size):
    samples = []
    for i in temp_array:
        samples.append(sample_generation(no_samples,i,lattice_size=lattice_size))
    samples = tf.concat(samples,axis=0)
    return samples

In [6]:
def magnetization(lattices):
    mag =  tf.math.sqrt(tf.reduce_mean(lattices[:,:,:,0],axis=[1,2])**2 +
                        tf.reduce_mean(lattices[:,:,:,1],axis=[1,2])**2)
    return mag

In [7]:
def magnetization_ycomp(lattices):
    y_comp = tf.reduce_mean(lattices[:,:,:,1],axis=[1,2])**2
    return y_comp

In [8]:
BATCH_SIZE = 256
num_classes = 32

xy_data = np.float32(data_loader.load_data_mh_generated('./data/data/8x8_gibbslattices.pkl'))/(2*np.pi)

trainset  = []
for i in range(num_classes):
    trainset.append(xy_data[10000*i:10000*i+5000])

trainset = np.reshape(np.array(trainset),(-1,l,l,1))

lattices1=[]
index_set = np.arange(num_classes)
for i in index_set:
    lattices1.append(xy_data[10000*i+5000:10000*i+6000])
lattices1=np.array(lattices1)
lattices1=tf.reshape(lattices1,(-1,l,l,1))



temp_val = np.float32(np.linspace(0.05,2.05,num_classes))
Temp = np.repeat(temp_val,5000)
T = np.repeat(Temp,l*l).reshape(-1,l,l,1)

cos_input = tf.math.cos(2*np.pi*trainset)
sin_input = tf.math.sin(2*np.pi*trainset)
training_inputs = tf.concat([cos_input,sin_input],axis=-1)

training_dataset = tf.data.Dataset.from_tensor_slices((training_inputs,T,Temp))
training_dataset = training_dataset.shuffle(buffer_size = 1024).batch(BATCH_SIZE)



2023-10-09 23:01:26.313315: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-10-09 23:01:26.313555: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: MADHAVLAB2
2023-10-09 23:01:26.313570: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: MADHAVLAB2
2023-10-09 23:01:26.313832: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 530.30.2
2023-10-09 23:01:26.313882: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 530.30.2
2023-10-09 23:01:26.313893: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 530.30.2


In [9]:
def generator_model():
    inputs = keras.Input(shape=(l,l,2))
    x      = layers.Conv2D(filters=8,kernel_size=(5,5),strides=1,padding='valid',activation='tanh')(periodic_padding(inputs,2))
    x      = layers.Conv2D(filters=16,kernel_size=(5,5),strides=1,padding='valid',activation='tanh')(periodic_padding(x,2))
    x      = layers.Conv2D(filters=32,kernel_size=(3,3),strides=1,padding='valid',activation='tanh')(periodic_padding(x,1))
    x      = layers.Conv2D(filters=2,kernel_size=(3,3),strides=1,padding='valid')(periodic_padding(x,1))
    x_cos  = x[:,:,:,0]/((x[:,:,:,0]**2 + x[:,:,:,1]**2)**0.5)
    x_sin  = x[:,:,:,1]/((x[:,:,:,0]**2 + x[:,:,:,1]**2)**0.5)
    x_vec  = tf.stack([x_cos,x_sin],axis=-1)
    model = keras.Model(inputs = inputs,outputs = x_vec,name='Generator_model')
    model.summary()
    return model

In [10]:
def discriminator_model():
    inputs = keras.Input(shape=(l,l,3))
    x  = layers.Conv2D(filters=8,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer1')(periodic_padding(inputs,1))
    x  = layers.Conv2D(filters=10,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer2')(x)
    x  = layers.Conv2D(filters=16,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer3')(x)
    x  = layers.Conv2D(filters=32,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer4')(x)
    x  = layers.Reshape((2*2*32,),name='layer5')(x)
    x  = layers.Dense(16,activation='tanh',name='layer6')(x)
    
    output = layers.Dense(1,name='layer7')(x)
    model  = keras.Model(inputs=inputs,outputs=output,name='Discriminator_Model')
    model.summary()
    return model

In [11]:
def auxiliary_model():
    inputs = keras.Input(shape=(l,l,2))
    x  = layers.Conv2D(filters=8,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer1')(periodic_padding(inputs,1))
    x  = layers.Conv2D(filters=10,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer2')(x)
    x  = layers.Conv2D(filters=16,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer3')(x)
    x  = layers.Conv2D(filters=32,kernel_size=(3,3),strides=1,padding='valid',activation='tanh',name='layer4')(x)
    x  = layers.Reshape((2*2*32,),name='layer5')(x)
    x  = layers.Dense(16,activation='tanh',name='layer6')(x)

    output = layers.Dense(2,name='layer11')(x)
    model  = keras.Model(inputs=inputs,outputs=output,name='Auxiliary_Model')
    model.summary()
    return model
    

In [12]:
generator = generator_model()
discriminator = discriminator_model()
auxiliary = auxiliary_model()
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output,fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output),real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output),fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output),fake_output)

def auxiliary_loss(temp,parameters):
    return 0.5*(tf.reduce_mean(((temp-parameters[:,0])**2)/tf.exp(parameters[:,1])
                        + 1.837 + parameters[:,1]))


Model: "Generator_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8, 8, 2)]    0           []                               
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 2, 2, 2)     0           ['input_1[0][0]']                
 ingOpLambda)                                                                                     
                                                                                                  
 tf.__operators__.getitem_1 (Sl  (None, 2, 8, 2)     0           ['input_1[0][0]']                
 icingOpLambda)                                                                                   
                                                                                    

                                                                 0]']                             
                                                                                                  
 tf.concat_5 (TFOpLambda)       (None, 8, 12, 8)     0           ['tf.__operators__.getitem_11[0][
                                                                 0]',                             
                                                                  'conv2d[0][0]',                 
                                                                  'tf.__operators__.getitem_12[0][
                                                                 0]']                             
                                                                                                  
 tf.concat_6 (TFOpLambda)       (None, 2, 12, 8)     0           ['tf.__operators__.getitem_13[0][
                                                                 0]',                             
          

 tf.__operators__.getitem_28 (S  (None, 8, 1, 32)    0           ['conv2d_2[0][0]']               
 licingOpLambda)                                                                                  
                                                                                                  
 tf.__operators__.getitem_29 (S  (None, 1, 1, 32)    0           ['conv2d_2[0][0]']               
 licingOpLambda)                                                                                  
                                                                                                  
 tf.__operators__.getitem_30 (S  (None, 1, 8, 32)    0           ['conv2d_2[0][0]']               
 licingOpLambda)                                                                                  
                                                                                                  
 tf.__operators__.getitem_31 (S  (None, 1, 1, 32)    0           ['conv2d_2[0][0]']               
 licingOpL

                                                                 0]',                             
                                                                  'tf.math.pow_5[0][0]']          
                                                                                                  
 tf.stack (TFOpLambda)          (None, 8, 8, 2)      0           ['tf.math.truediv[0][0]',        
                                                                  'tf.math.truediv_1[0][0]']      
                                                                                                  
Total params: 8,842
Trainable params: 8,842
Non-trainable params: 0
__________________________________________________________________________________________________
Model: "Discriminator_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (In

                                                                                                  
 tf.__operators__.getitem_46 (S  (None, 1, 1, 2)     0           ['input_3[0][0]']                
 licingOpLambda)                                                                                  
                                                                                                  
 tf.__operators__.getitem_47 (S  (None, 1, 8, 2)     0           ['input_3[0][0]']                
 licingOpLambda)                                                                                  
                                                                                                  
 tf.__operators__.getitem_48 (S  (None, 1, 1, 2)     0           ['input_3[0][0]']                
 licingOpLambda)                                                                                  
                                                                                                  
 tf.__oper

In [48]:
lambda1 = 10.0
lambda2 = 1.0

boundaries = [1250*1, 1250*2, 1250*3, 1250*4, 1250*5, 1250*6, 1250*7, 1250*8, 1250*9, 1250*10]
 
gen_lr = 5.e-5
dis_lr = 5.e-5
aux_lr = 5.e-7
decay  = 0.95

values_gen = [gen_lr, gen_lr*decay, gen_lr*(decay)**2, gen_lr*(decay)**3, gen_lr*(decay)**4,
              gen_lr*(decay)**5, gen_lr*(decay)**6, gen_lr*(decay)**7, gen_lr*(decay)**8, 
              gen_lr*(decay)**9, gen_lr*(decay)**10]
values_dis = [dis_lr, dis_lr*decay, dis_lr*(decay)**2, dis_lr*(decay)**3, dis_lr*(decay)**4,
              dis_lr*(decay)**5, dis_lr*(decay)**6, dis_lr*(decay)**7, dis_lr*(decay)**8, 
              dis_lr*(decay)**9, dis_lr*(decay)**10]
values_aux = [aux_lr, aux_lr*decay, aux_lr*(decay)**2, aux_lr*(decay)**3, aux_lr*(decay)**4,
              aux_lr*(decay)**5, aux_lr*(decay)**6, aux_lr*(decay)**7, aux_lr*(decay)**8, 
              aux_lr*(decay)**9, aux_lr*(decay)**10]
learning_rate_fn_gen = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values_gen)
learning_rate_fn_dis = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values_dis)
learning_rate_fn_aux = keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values_aux)

generator_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn_gen)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn_dis)
auxiliary_model_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn_aux)

train_G_loss = tf.keras.metrics.Mean('train_G_loss', dtype=tf.float32)
train_D_loss = tf.keras.metrics.Mean('train_D_loss', dtype=tf.float32)
train_mag_loss = tf.keras.metrics.Mean('mean_mag_ycomp',dtype=tf.float32)
train_entropy_loss = tf.keras.metrics.Mean('train_Entropy_loss',dtype=tf.float32)
train_net_gen_loss = tf.keras.metrics.Mean('total_Gen_loss',dtype=tf.float32)

In [44]:
checkpoint_directory = './igan/8x8/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_directory, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 auxiliary_model_optimizer=auxiliary_model_optimizer,
                                 generator = generator,discriminator= discriminator,auxiliary=auxiliary)

In [45]:
#checkpoint.restore(checkpoint_prefix)

In [49]:
@tf.function
def train_step(x,y,temp):
    zrand = tf.random.normal(shape=tf.shape(y),mean=0.0,stddev=1.0,dtype=tf.dtypes.float32)
    noise = tf.concat([zrand,y],axis=-1)
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape , tf.GradientTape() as aux_tape:
        gen_data = tf.cast(generator(noise,training = True),dtype=tf.float32)
        input1 = tf.concat([x,y],axis = -1)
        input2 = tf.concat([gen_data,y],axis = -1)
        real_output = discriminator(input1,training=True)
        fake_output = discriminator(input2,training=True)
        input3 = tf.concat([x,gen_data],axis=0)
        temp_inp3 = tf.cast(tf.concat([temp, temp],axis=0),dtype=tf.float32)
        T_parameters = auxiliary(input3,training=True)
        
        mag_ycomp = tf.cast(tf.reduce_mean(magnetization_ycomp(gen_data)),dtype=tf.float32)
        gen_loss = generator_loss(fake_output)
        aux_loss = auxiliary_loss(temp_inp3,T_parameters)
        
        disc_loss = discriminator_loss(real_output,fake_output)
        net_gen_loss = gen_loss + lambda1*mag_ycomp + lambda2*aux_loss
        
    gradients_of_generator = gen_tape.gradient(net_gen_loss,generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss,discriminator.trainable_variables)
    gradients_of_auxiliary = aux_tape.gradient(aux_loss,auxiliary.trainable_variables)
    
    generator_optimizer.apply_gradients(zip(gradients_of_generator,generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator,discriminator.trainable_variables))
    auxiliary_model_optimizer.apply_gradients(zip(gradients_of_auxiliary,auxiliary.trainable_variables))
    train_G_loss(gen_loss)
    train_D_loss(disc_loss)
    train_mag_loss(mag_ycomp)
    train_entropy_loss(aux_loss)
    train_net_gen_loss(net_gen_loss)

In [ ]:
EPOCH = 50
start = time.time()
for epoch in range(EPOCH):
    
    for step,(x,y,temp) in enumerate(training_dataset):
        train_step(x,y,temp)
    
   
    template = 'Epoch {:2d}, Total_Gen_Loss: {:.7f}, Gen_Loss: {:.7f}, Disc_Loss: {:.7f} , Mag_Loss: {:.10f} , Entropy_loss: {:.10f}'
    print (template.format(epoch+1,train_net_gen_loss.result(),train_G_loss.result(),train_D_loss.result(),
                           train_mag_loss.result() , train_entropy_loss.result()))

    # Reset metrics every epoch
    train_G_loss.reset_states()
    train_D_loss.reset_states() 
    train_mag_loss.reset_states()
    train_entropy_loss.reset_states()
    train_net_gen_loss.reset_states()
    
    
stop = time.time()
print('Time: ', stop - start)
            

Epoch  1, Total_Gen_Loss: 1.7401979, Gen_Loss: 1.5084617, Disc_Loss: 1.0307680 , Mag_Loss: 0.0216413382 , Entropy_loss: 0.0153211672
Epoch  2, Total_Gen_Loss: 1.4661701, Gen_Loss: 1.3390707, Disc_Loss: 0.8728279 , Mag_Loss: 0.0123217199 , Entropy_loss: 0.0038821457
Epoch  3, Total_Gen_Loss: 1.5638895, Gen_Loss: 1.3756185, Disc_Loss: 0.8553162 , Mag_Loss: 0.0208530854 , Entropy_loss: -0.0202602502
Epoch  4, Total_Gen_Loss: 1.7344913, Gen_Loss: 1.4124060, Disc_Loss: 0.9223102 , Mag_Loss: 0.0328983814 , Entropy_loss: -0.0068986220
Epoch  5, Total_Gen_Loss: 1.6790537, Gen_Loss: 1.3978468, Disc_Loss: 0.9398233 , Mag_Loss: 0.0270383023 , Entropy_loss: 0.0108234938
Epoch  6, Total_Gen_Loss: 1.6656744, Gen_Loss: 1.4281770, Disc_Loss: 0.8286834 , Mag_Loss: 0.0259352457 , Entropy_loss: -0.0218556747
Epoch  7, Total_Gen_Loss: 1.6541736, Gen_Loss: 1.4538956, Disc_Loss: 0.8336142 , Mag_Loss: 0.0232690889 , Entropy_loss: -0.0324128643
Epoch  8, Total_Gen_Loss: 1.5751395, Gen_Loss: 1.4441890, Disc_Lo

In [ ]:
checkpoint.save(file_prefix = checkpoint_prefix)

In [ ]:
samples=samples_generation_for_all_temp(1000,temp_val,8)

In [ ]:
comparison_plot(lattices1,samples,1000,0.05,2.05,num_classes,J=1,K=0,name='./igan/8x8/comparison_plot_ckpt_')

In [ ]:
evaluation_metrics(lattices1,samples,0.05,2.05,num_classes,J=1,K=0)

In [ ]:
output = open('./igan/8x8/lattices_ckpt_.pkl', 'wb')
pickle.dump(samples, output)
output.close()